In [22]:
from torchvision.datasets import ImageFolder
from loading.datasets import CustomLabelEncoder
from loading.loaders import SetSpec, get_data_for_spec, get_classes, make_image_loader
import os
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import plotly.express as px
import torchvision.models as models
import ast
from torchvision.transforms import transforms

In [23]:

image_net_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

class_dictionary = {}
with open("imagenet/image_idx_to_label.txt", "r") as f:
    contents= f.read()
    class_dictionary = ast.literal_eval(contents)


In [24]:
mnt_path = '/mnt/chansey/'
umc_data_path = os.path.join(mnt_path, 'klaus/data/devices/')
umc_img_root = os.path.join(mnt_path, 'klaus/total_patients/')


desired_set_specs = {'source_train': [SetSpec('ESAOTE_6100', 'umc', 'train', umc_data_path, umc_img_root)],
                    'target_train': [SetSpec('Philips_iU22', 'umc', 'train', umc_data_path, umc_img_root)]}
attribute = 'Muscle'
is_multi = False
use_one_channel = False
normalizer_name = 'ESAOTE_6100'
batch_size = 8
train_classes = None
is_classification = True
label_encoder = None
set_loaders = {}
for set_name, set_spec_list in desired_set_specs.items():

    loaders = []
    for set_spec in set_spec_list:
        print(set_spec)
        # pass the classes in to ensure that only those are present in all the sets
        data = get_data_for_spec(set_spec, loader_type='image', attribute=attribute, class_values=train_classes)
        print(f'Loaded {len(data)} elements.')

        # if classification and this is the train set, we want to fit the label encoder on this
        if is_classification & (set_name == 'source_train'):
            train_classes = get_classes(data, attribute)
            print(train_classes)
            label_encoder = CustomLabelEncoder(train_classes, one_hot_encode=False)
        print(get_classes(data, attribute))
        img_path = set_spec.img_root_path
        loader = make_image_loader(data, img_path, use_one_channel, normalizer_name,
                                       attribute, batch_size, set_spec.device, limit_image_size=True,
                                       label_encoder=label_encoder, pin_memory=False, is_multi=is_multi)

        loaders.append(loader)

    set_loaders[set_name] = loaders

ESAOTE_6100/train
Found a total of 82258 images.
Retained 82258 images after dropping null values.
Loaded 82258 elements.
Biceps brachii                9049
Tibialis anterior             8960
Gastrocnemius medial head     8259
Flexor carpi radialis         8232
Vastus lateralis              5962
Rectus femoris                5190
Deltoideus                    4963
Interosseus dorsalis I        4631
Rectus abdominis              3475
Trapezius                     3402
Masseter                      3020
Digastricus                   2994
Sterno cleido                 2782
Flexor digitorum profundus    2328
Peroneus tertius              2194
Geniohyoideus                 1611
Depressor anguli oris          681
Zygomaticus major              519
Temporalis                     509
Orbicularis oris               441
Extensors underarm             435
Rectus femoris prox            329
Vastus lateralis prox          322
Mentalis                       315
Biceps femoris                 220
Ili

In [25]:
model = models.resnet18(pretrained=True)
model.to('cuda')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
def get_n_images_from_loader(n, loader, name):
    print(name)
    esaote_images = []
    esaote_labels = []
    all_outputs = []

    for images, labels in loader:
        esaote_images.extend(images.cpu())
        outputs = model(images.to('cuda')).detach().cpu().numpy()
        all_outputs.extend(outputs)
        if hasattr(loader.dataset, 'targets'):
            labels = [loader.dataset.classes[int(label)] for label in labels]
        elif hasattr(loader.dataset, 'label_encoder'):
            labels = [loader.dataset.label_encoder.classes[int(label)] for label in labels]
        esaote_labels.extend(labels)
        if len(esaote_images) > n:
            break
            
    esaote_images = [elem[0].squeeze().flatten().numpy() for elem in esaote_images]
            
    return esaote_images, esaote_labels, [name] * len(esaote_images), all_outputs

n_images_per_set = 100
esaote_loader = set_loaders['source_train'][0]
philips_loader = set_loaders['target_train'][0]


e_images, e_labels, e_names, e_outputs = get_n_images_from_loader(n_images_per_set, esaote_loader, 'esaote')
p_images, p_labels, p_names, p_outputs = get_n_images_from_loader(n_images_per_set, philips_loader, 'philips')

imagenet
esaote
philips


In [44]:
imagenet_sample = ImageFolder('~/Thesis_2/ImageNet-Datasets-Downloader/images/imagenet_images', transform=image_net_transform)
imagenet_loader = DataLoader(imagenet_sample, num_workers=8, batch_size=batch_size)
i_images, i_labels, i_names, i_outputs = get_n_images_from_loader(n_images_per_set, imagenet_loader, 'imagenet')

imagenet
140


In [45]:
all_images = np.concatenate([i_images, e_images, p_images])
sources = np.concatenate([i_names, e_names, p_names])
classes = np.concatenate([i_labels, e_labels, p_labels])
outputs = np.concatenate([i_outputs, e_outputs, p_outputs])

In [46]:
tsne = TSNE(n_iter=1000, learning_rate=10, perplexity=5)
pca = PCA()
fit_coordinates = pca.fit_transform(outputs)

In [47]:
d = {'x': fit_coordinates[:,0], 'y' : fit_coordinates[:,1], 'source': sources, 'class': classes}
tsne_frame = pd.DataFrame(d)

In [48]:
px.scatter(tsne_frame,x='x', y='y',color='source')


In [49]:
class_dict = get_imagenet_class_dict()
labels = []
for output in outputs:
    max_ind = output.argmax()
    label = class_dictionary[max_ind]
    labels.append(label)
tsne_frame['labels'] = labels

In [ ]:
tsne_frame.groupby(['source','labels']).size().nlargest(25)

In [51]:
from torch import softmax
px.bar(outputs[0])

source    labels                                                                             
philips   maze, labyrinth                                                                        89
esaote    nematode, nematode worm, roundworm                                                     67
philips   fountain                                                                               54
          nematode, nematode worm, roundworm                                                     48
esaote    thimble                                                                                37
          toaster                                                                                28
          saltshaker, salt shaker                                                                26
imagenet  great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias    26
          tench, Tinca tinca                                                                     26
      

In [2]:
from torch import softmax
px.bar(outputs[0])

NameError: name 'outputs' is not defined